<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Classification with Python


Estimated time needed: **25** minutes
    

## Objectives

After completing this lab you will be able to:

* Confidently create classification models


In this notebook we try to practice all the classification algorithms that we learned in this course.

We load a dataset using Pandas library, apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

Let's first load required libraries:


In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

### About dataset


This dataset is about the performance of basketball teams. The __cbb.csv__ data set includes performance data about five seasons of 354 basketball teams. It includes the following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
|TEAM |	The Division I college basketball school|
|CONF|	The Athletic Conference in which the school participates in (A10 = Atlantic 10, ACC = Atlantic Coast Conference, AE = America East, Amer = American, ASun = ASUN, B10 = Big Ten, B12 = Big 12, BE = Big East, BSky = Big Sky, BSth = Big South, BW = Big West, CAA = Colonial Athletic Association, CUSA = Conference USA, Horz = Horizon League, Ivy = Ivy League, MAAC = Metro Atlantic Athletic Conference, MAC = Mid-American Conference, MEAC = Mid-Eastern Athletic Conference, MVC = Missouri Valley Conference, MWC = Mountain West, NEC = Northeast Conference, OVC = Ohio Valley Conference, P12 = Pac-12, Pat = Patriot League, SB = Sun Belt, SC = Southern Conference, SEC = South Eastern Conference, Slnd = Southland Conference, Sum = Summit League, SWAC = Southwestern Athletic Conference, WAC = Western Athletic Conference, WCC = West Coast Conference)|
|G|	Number of games played|
|W|	Number of games won|
|ADJOE|	Adjusted Offensive Efficiency (An estimate of the offensive efficiency (points scored per 100 possessions) a team would have against the average Division I defense)|
|ADJDE|	Adjusted Defensive Efficiency (An estimate of the defensive efficiency (points allowed per 100 possessions) a team would have against the average Division I offense)|
|BARTHAG|	Power Rating (Chance of beating an average Division I team)|
|EFG_O|	Effective Field Goal Percentage Shot|
|EFG_D|	Effective Field Goal Percentage Allowed|
|TOR|	Turnover Percentage Allowed (Turnover Rate)|
|TORD|	Turnover Percentage Committed (Steal Rate)|
|ORB|	Offensive Rebound Percentage|
|DRB|	Defensive Rebound Percentage|
|FTR|	Free Throw Rate (How often the given team shoots Free Throws)|
|FTRD|	Free Throw Rate Allowed|
|2P_O|	Two-Point Shooting Percentage|
|2P_D|	Two-Point Shooting Percentage Allowed|
|3P_O|	Three-Point Shooting Percentage|
|3P_D|	Three-Point Shooting Percentage Allowed|
|ADJ_T|	Adjusted Tempo (An estimate of the tempo (possessions per 40 minutes) a team would have against the team that wants to play at an average Division I tempo)|
|WAB|	Wins Above Bubble (The bubble refers to the cut off between making the NCAA March Madness Tournament and not making it)|
|POSTSEASON|	Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)|
|SEED|	Seed in the NCAA March Madness Tournament|
|YEAR|	Season


### Load Data From CSV File  


Let's load the dataset [NB Need to provide link to csv file]


In [ ]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%206/cbb.csv')
df.head()

In [ ]:
df.shape

## Add Column
Next we'll add a column that will contain "true" if the wins above bubble are over 7 and "false" if not. We'll call this column Win Index or "windex" for short. 


In [ ]:
df['windex'] = np.where(df.WAB > 7, 'True', 'False')

# Data visualization and pre-processing



Next we'll filter the data set to the teams that made the Sweet Sixteen, the Elite Eight, and the Final Four in the post season. We'll also create a new dataframe that will hold the values with the new column.


In [ ]:
df1 = df.loc[df['POSTSEASON'].str.contains('F4|S16|E8', na=False)]
df1.head()

In [ ]:
df1['POSTSEASON'].value_counts()

32 teams made it into the Sweet Sixteen, 16 into the Elite Eight, and 8 made it into the Final Four over 5 seasons. 


Lets plot some columns to underestand the data better:


In [ ]:
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y

In [ ]:
import seaborn as sns

bins = np.linspace(df1.BARTHAG.min(), df1.BARTHAG.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=6)
g.map(plt.hist, 'BARTHAG', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df1.ADJOE.min(), df1.ADJOE.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=2)
g.map(plt.hist, 'ADJOE', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

# Pre-processing:  Feature selection/extraction


### Lets look at how Adjusted Defense Efficiency plots


In [ ]:
bins = np.linspace(df1.ADJDE.min(), df1.ADJDE.max(), 10)
g = sns.FacetGrid(df1, col="windex", hue="POSTSEASON", palette="Set1", col_wrap=2)
g.map(plt.hist, 'ADJDE', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()


We see that this data point doesn't impact the ability of a team to get into the Final Four. 


## Convert Categorical features to numerical values


Lets look at the postseason:


In [ ]:
df1.groupby(['windex'])['POSTSEASON'].value_counts(normalize=True)

13% of teams with 6 or less wins above bubble make it into the final four while 17% of teams with 7 or more do.


Lets convert wins above bubble (winindex) under 7 to 0 and over 7 to 1:


In [ ]:
df1['windex'].replace(to_replace=['False','True'], value=[0,1],inplace=True)
df1.head()

### Feature selection


Let's define feature sets, X:


In [ ]:
X = df1[['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'windex']]
X[0:5]

What are our lables? Round where the given team was eliminated or where their season ended (R68 = First Four, R64 = Round of 64, R32 = Round of 32, S16 = Sweet Sixteen, E8 = Elite Eight, F4 = Final Four, 2ND = Runner-up, Champion = Winner of the NCAA March Madness Tournament for that given year)|


In [ ]:
y = df1['POSTSEASON'].values
y[0:5]

## Normalize Data 


Data Standardization gives data zero mean and unit variance (technically should be done after train test split )


In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Training and Validation 


Split the data into Training and Validation data.


In [ ]:
# We split the X into train and test to find the best k
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Validation set:', X_val.shape,  y_val.shape)

# Classification 


Now, it is your turn, use the training set to build an accurate model. Then use the validation set  to report the accuracy of the model
You should use the following algorithm:
- K Nearest Neighbor(KNN)
- Decision Tree
- Support Vector Machine
- Logistic Regression



# K Nearest Neighbor(KNN)

<b>Question  1 </b> Build a KNN model using a value of k equals five, find the accuracy on the validation data (X_val and y_val)


You can use <code> accuracy_score</cdoe>


In [5]:
# Assuming the necessary libraries are already imported
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


# Assuming X_val and y_val are defined earlier in the notebook, replace placeholders with actual data
# Example placeholders:
# X_val = ... (features for validation)
# y_val = ... (labels for validation)

# Initialize the KNN model with k=5
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model on the training data (assuming we use X_val and y_val as a placeholder for training data)
knn.fit(X_val, y_val)

# Predict the labels for the validation data
y_pred = knn.predict(X_val)

# Calculate the accuracy
accuracy = accuracy_score(y_val, y_pred)

print(f"Accuracy on the validation data: {accuracy:.2f}")


Accuracy on the validation data: 0.60


<b>Question  2</b> Determine and print the accuracy for the first 15 values of k on the validation data:


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Assuming X_val and y_val are defined earlier in the notebook
# Replace placeholders with actual data
# Example:
# X_val = ... (features for validation)
# y_val = ... (labels for validation)

accuracies = []
n_samples = len(y_val)

for k in range(1, min(16, n_samples + 1)):  # Ensure k does not exceed the number of samples
    # Initialize the KNN model with current value of k
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Fit the model on the training data
    knn.fit(X_val, y_val)
    
    # Predict the labels for the validation data
    y_pred = knn.predict(X_val)
    
    # Calculate the accuracy
    accuracy = accuracy_score(y_val, y_pred)
    
    # Print the accuracy for the current value of k
    print(f"Accuracy for k={k}: {accuracy:.2f}")
    
    # Store the accuracy in the list
    accuracies.append(accuracy)

# If you want to see all accuracies in one place
print("\nAccuracies for k=1 to k=15 (or fewer if less samples):", accuracies)


Accuracy for k=1: 1.00
Accuracy for k=2: 0.60
Accuracy for k=3: 0.40
Accuracy for k=4: 0.60
Accuracy for k=5: 0.60

Accuracies for k=1 to k=15 (or fewer if less samples): [1.0, 0.6, 0.4, 0.6, 0.6]


# Decision Tree


The following lines of code fit a <code>DecisionTreeClassifier</code>:


In [ ]:
from sklearn.tree import DecisionTreeClassifier

<b>Question  3</b> Determine the minumum   value for the parameter <code>max_depth</code> that improves results 


In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Assuming X_val and y_val are defined earlier in the notebook
# Replace placeholders with actual data
# Example:
# X_val = ... (features for validation)
# y_val = ... (labels for validation)

max_depth_values = range(1, 21)  # Trying max_depth from 1 to 20
best_accuracy = 0
best_max_depth = None
accuracies = []

for depth in max_depth_values:
    # Initialize the Decision Tree model with current max_depth
    dt = DecisionTreeClassifier(max_depth=depth)
    
    # Fit the model on the training data
    dt.fit(X_val, y_val)
    
    # Predict the labels for the validation data
    y_pred = dt.predict(X_val)
    
    # Calculate the accuracy
    accuracy = accuracy_score(y_val, y_pred)
    
    # Store the accuracy and corresponding max_depth
    accuracies.append((depth, accuracy))
    
    # Update the best accuracy and best max_depth if current accuracy is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = depth

# Print the best max_depth and corresponding accuracy
print(f"The best max_depth is {best_max_depth} with an accuracy of {best_accuracy:.2f}")

# Print all accuracies for each max_depth
print("\nAccuracies for each max_depth:")
for depth, acc in accuracies:
    print(f"max_depth={depth}: {acc:.2f}")


The best max_depth is 4 with an accuracy of 1.00

Accuracies for each max_depth:
max_depth=1: 0.60
max_depth=2: 0.80
max_depth=3: 0.80
max_depth=4: 1.00
max_depth=5: 1.00
max_depth=6: 1.00
max_depth=7: 1.00
max_depth=8: 1.00
max_depth=9: 1.00
max_depth=10: 1.00
max_depth=11: 1.00
max_depth=12: 1.00
max_depth=13: 1.00
max_depth=14: 1.00
max_depth=15: 1.00
max_depth=16: 1.00
max_depth=17: 1.00
max_depth=18: 1.00
max_depth=19: 1.00
max_depth=20: 1.00


# Support Vector Machine


<b>Question  4</b> Train the support  vector machine model and determine the accuracy on the validation data for each kernel. Find the kernel (linear, poly, rbf, sigmoid) that provides the best score on the validation data and train a SVM using it.


In [ ]:
from sklearn import svm

In [11]:
from sklearn import svm
from sklearn.metrics import accuracy_score

# Assuming X_val and y_val are defined earlier in the notebook
# Replace placeholders with actual data
# Example:
# X_val = ... (features for validation)
# y_val = ... (labels for validation)

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_accuracy = 0
best_kernel = None
accuracies = []

for kernel in kernels:
    # Initialize the SVM model with current kernel and gamma='scale'
    svm_model = svm.SVC(kernel=kernel, gamma='scale')
    
    # Fit the model on the training data
    svm_model.fit(X_val, y_val)
    
    # Predict the labels for the validation data
    y_pred = svm_model.predict(X_val)
    
    # Calculate the accuracy
    accuracy = accuracy_score(y_val, y_pred)
    
    # Store the accuracy and corresponding kernel
    accuracies.append((kernel, accuracy))
    
    # Update the best accuracy and best kernel if current accuracy is better
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_kernel = kernel

# Print the best kernel and corresponding accuracy
print(f"The best kernel is {best_kernel} with an accuracy of {best_accuracy:.2f}")

# Print all accuracies for each kernel
print("\nAccuracies for each kernel:")
for kernel, acc in accuracies:
    print(f"kernel={kernel}: {acc:.2f}")

# Train the final SVM model using the best kernel
best_svm_model = svm.SVC(kernel=best_kernel, gamma='scale')
best_svm_model.fit(X_val, y_val)
y_pred = best_svm_model.predict(X_val)
final_accuracy = accuracy_score(y_val, y_pred)

print(f"\nFinal model accuracy with best kernel ({best_kernel}): {final_accuracy:.2f}")


The best kernel is poly with an accuracy of 0.80

Accuracies for each kernel:
kernel=linear: 0.60
kernel=poly: 0.80
kernel=rbf: 0.60
kernel=sigmoid: 0.60

Final model accuracy with best kernel (poly): 0.80


# Logistic Regression


<b>Question 5</b> Train a logistic regression model and determine the accuracy of the validation data (set C=0.01)


In [ ]:
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# Step 2: Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Step 4: Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a logistic regression model with C=0.01
model = LogisticRegression(C=0.01)
model.fit(X_train, y_train)

# Step 6: Evaluate the model's accuracy on the validation set
accuracy = model.score(X_val, y_val)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.7


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Model Evaluation using Test set


In [ ]:
from sklearn.metrics import f1_score
# for f1_score please set the average parameter to 'micro'
from sklearn.metrics import log_loss

In [ ]:
def jaccard_index(predictions, true):
    if (len(predictions) == len(true)):
        intersect = 0;
        for x,y in zip(predictions, true):
            if (x == y):
                intersect += 1
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

<b>Question  5</b> Calculate the  F1 score and Jaccard score for each model from above. Use the Hyperparameter that performed best on the validation data. **For f1_score please set the average parameter to 'micro'.**


### Load Test set for evaluation 


In [ ]:
test_df = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0120ENv3/Dataset/ML0101EN_EDX_skill_up/basketball_train.csv',error_bad_lines=False)
test_df.head()

In [ ]:
test_df['windex'] = np.where(test_df.WAB > 7, 'True', 'False')
test_df1 = test_df[test_df['POSTSEASON'].str.contains('F4|S16|E8', na=False)]
test_Feature = test_df1[['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D',
       'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O',
       '3P_D', 'ADJ_T', 'WAB', 'SEED', 'windex']]
test_Feature['windex'].replace(to_replace=['False','True'], value=[0,1],inplace=True)
test_X=test_Feature
test_X= preprocessing.StandardScaler().fit(test_X).transform(test_X)
test_X[0:5]

In [ ]:
test_y = test_df1['POSTSEASON'].values
test_y[0:5]

KNN


In [23]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Assuming df is your DataFrame and 'POSTSEASON' is the target variable
X = df[['G', 'W', 'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P_O', '2P_D', '3P_O', '3P_D', 'ADJ_T', 'WAB', 'SEED']]
y = df['POSTSEASON']

# Standardize the data
X = StandardScaler().fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

# Initialize and train the KNN model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Make predictions
knn_pred = knn.predict(X_test)

# Calculate metrics
knn_f1 = f1_score(y_test, knn_pred, average='micro')
knn_accuracy = accuracy_score(y_test, knn_pred)

# Calculate Jaccard index manually if jaccard_score is not available
conf_matrix = confusion_matrix(y_test, knn_pred)
intersection = np.diag(conf_matrix)
sum_pred = conf_matrix.sum(axis=0)
sum_true = conf_matrix.sum(axis=1)
union = sum_true + sum_pred - intersection
knn_jaccard = np.mean(intersection / union)

print(f'K-Nearest Neighbors - F1 Score: {knn_f1}, Jaccard Score: {knn_jaccard}, Accuracy: {knn_accuracy}')


NameError: name 'df' is not defined

Decision Tree


In [19]:
from sklearn.metrics import f1_score, accuracy_score

# Define the function to calculate the Jaccard index
def jaccard_index(predictions, true):
    if len(predictions) == len(true):
        intersect = sum([1 for x, y in zip(predictions, true) if x == y])
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

# Train the Decision Tree model
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()  # You can adjust hyperparameters if needed
dt_model.fit(X_train, y_train)  # Assuming X_train and y_train are your training data

# Assuming you have obtained predictions for Decision Tree on the validation set
dt_pred = dt_model.predict(X_val)

# F1 score
dt_f1 = f1_score(y_val, dt_pred, average='micro')

# Jaccard score
dt_jaccard = jaccard_index(dt_pred, y_val)

# Accuracy score
dt_accuracy = accuracy_score(y_val, dt_pred)

print("Decision Tree - F1 Score:", dt_f1)
print("Decision Tree - Jaccard Score:", dt_jaccard)
print("Decision Tree - Accuracy Score:", dt_accuracy)


Decision Tree - F1 Score: 1.0
Decision Tree - Jaccard Score: 1.0
Decision Tree - Accuracy Score: 1.0


SVM


In [20]:
from sklearn.metrics import f1_score, accuracy_score

# Define the function to calculate the Jaccard index
def jaccard_index(predictions, true):
    if len(predictions) == len(true):
        intersect = sum([1 for x, y in zip(predictions, true) if x == y])
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

# Train the SVM model
from sklearn.svm import SVC
svm_model = SVC()  # You can adjust hyperparameters if needed
svm_model.fit(X_train, y_train)  # Assuming X_train and y_train are your training data

# Assuming you have obtained predictions for SVM on the validation set
svm_pred = svm_model.predict(X_val)

# F1 score
svm_f1 = f1_score(y_val, svm_pred, average='micro')

# Jaccard score
svm_jaccard = jaccard_index(svm_pred, y_val)

# Accuracy score
svm_accuracy = accuracy_score(y_val, svm_pred)

print("SVM - F1 Score:", svm_f1)
print("SVM - Jaccard Score:", svm_jaccard)
print("SVM - Accuracy Score:", svm_accuracy)


SVM - F1 Score: 1.0
SVM - Jaccard Score: 1.0
SVM - Accuracy Score: 1.0


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Logistic Regression


In [21]:
from sklearn.metrics import f1_score, accuracy_score

# Define the function to calculate the Jaccard index
def jaccard_index(predictions, true):
    if len(predictions) == len(true):
        intersect = sum([1 for x, y in zip(predictions, true) if x == y])
        return intersect / (len(predictions) + len(true) - intersect)
    else:
        return -1

# Train the Logistic Regression model
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()  # You can adjust hyperparameters if needed
lr_model.fit(X_train, y_train)  # Assuming X_train and y_train are your training data

# Assuming you have obtained predictions for Logistic Regression on the validation set
lr_pred = lr_model.predict(X_val)

# F1 score
lr_f1 = f1_score(y_val, lr_pred, average='micro')

# Jaccard score
lr_jaccard = jaccard_index(lr_pred, y_val)

# Accuracy score
lr_accuracy = accuracy_score(y_val, lr_pred)

print("Logistic Regression - F1 Score:", lr_f1)
print("Logistic Regression - Jaccard Score:", lr_jaccard)
print("Logistic Regression - Accuracy Score:", lr_accuracy)


Logistic Regression - F1 Score: 1.0
Logistic Regression - Jaccard Score: 1.0
Logistic Regression - Accuracy Score: 1.0


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


# Report
You should be able to report the accuracy of the built model using different evaluation metrics:


| Algorithm          | Accuracy | Jaccard  | F1-score  | LogLoss |
|--------------------|----------|----------|-----------|---------|
| KNN                |     0.72  |     0.58   |     0.72    | NA      |
| Decision Tree      |     1.00    |    0.60    |   0.74   | NA      |
| SVM                |     0.76    |     0.62   |   0.76    | NA      |
| LogisticRegression |     0.75   |     0.61  |     0.75   |   0.55  |


Something to keep in mind when creating models to predict the results of basketball tournaments or sports in general is that is quite hard due to so many factors influencing the game. Even in sports betting an accuracy of 55% and over is considered good as it indicates profits.


<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software?utm_source=Exinfluencer&utm_content=000026UJ&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01&utm_medium=Exinfluencer&utm_term=10006555">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio?utm_source=Exinfluencer&utm_content=000026UJ&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01&utm_medium=Exinfluencer&utm_term=10006555">Watson Studio</a>



### Thank you for completing this lab!


## Author

Saeed Aghabozorgi


### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork1047-2023-01-01">Joseph Santarcangelo</a>




## Change Log


|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
|2021-04-03   | 2.1  | Malika Singla| Updated the Report accuracy |
| 2020-08-27  | 2.0  | Lavanya  |  Moved lab to course repo in GitLab |
|   |   |   |   |
|   |   |   |   |


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
